In [334]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt

csv.field_size_limit(1000000000000000000)

1000000000000000000

In [335]:
MASDEV = pd.read_pickle("./3_MASTER_DEVICE_File_wSTDMANUFAC.pkl")
MASDEV.shape

(430858, 30)

In [336]:
# columns that can be used for product identifier, as found in the ASSIGN_STANDARDIZED_MANUFACTURER file: 
# BRAND_NAME, GENERIC_NAME, MODEL_NUMBER

In [337]:
# checking the standard manufacturer indicator
MASDEV["STD_MANUFACTURER"].value_counts()

ST. JUDE             142433
MEDTRONIC            136584
BOSTON SCIENTIFIC    116823
BIOTRONIK             17189
IPG MFG                6111
ABBOTT                 4541
SORIN                  4137
CAMERON HEALTH         2540
ANGEION                 254
OSCOR                   246
Name: STD_MANUFACTURER, dtype: int64

In [338]:
# checking the columns
MASDEV['BRAND_NAME'].value_counts()

SPRINT QUATTRO SECURE                               24274
ENDOTAK RELIANCE                                    22790
DURATA STS OPTIM ACTIVE FIXATION, DF-4 CONNECTOR    16350
SPRINT QUATTRO SECURE S                             14535
SPRINT FIDELIS                                      14286
                                                    ...  
128478HINSYNC MAXIMO                                    1
MEDTRONIC DEFIBRILLATOR IMPLANT                         1
ST. JUDE MEDICAL 7122 DURATA                            1
RAPIDO CUTTER CATHETER                                  1
IFORIA 7 VR-T DX                                        1
Name: BRAND_NAME, Length: 3099, dtype: int64

In [339]:
MASDEV['GENERIC_NAME'].value_counts()

IMPLANTABLE CARDIOVERTER DEFIBRILLATOR               121757
DEFIBRILLATOR, AUTOMATIC IMPLANTABLE CARDIOVERTER     62044
DEFIBRILLATION LEAD                                   57557
IMPLANTABLE LEAD                                      31762
IMPLANTABLE TACHY LEAD                                27200
                                                      ...  
AICD V-LEAD                                               1
DEFIBRILLATGION LEAD                                      1
PACEMAKER/DEFIBRILLATOR, BIVENTRICULAR                    1
AICD HEART PACEMAKER                                      1
SPRINT FIDELIS VENTRICULAR LEAD WIRE                      1
Name: GENERIC_NAME, Length: 1398, dtype: int64

In [340]:
MASDEV['MODEL_NUMBER'].value_counts()

6947        10349
6935M62      9596
694765       8722
6949         8411
0185         7354
            ...  
365000          1
V-186HVS        1
V-186V3         1
429564          1
72210           1
Name: MODEL_NUMBER, Length: 2902, dtype: int64

In [341]:
# create the standardized device identifier column
MASDEV["STD_DEVICE"] = MASDEV['BRAND_NAME']
MASDEV["RECALL_CLASS"] = MASDEV['BRAND_NAME']
MASDEV = MASDEV.assign(RECALL_CLASS="0")
MASDEV.head()

,MDR_REPORT_KEY,REPORT_NUMBER,REPORT_SOURCE_CODE,MANUFACTURER_LINK_FLAG_,ADVERSE_EVENT_FLAG,PRODUCT_PROBLEM_FLAG,DATE_OF_EVENT,REPORTER_OCCUPATION_CODE,HEALTH_PROFESSIONAL,INITIAL_REPORT_TO_FDA,...,GENERIC_NAME,DEVICE_OPERATOR,MODEL_NUMBER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,year,STD_MANUFACTURER,STD_DEVICE,RECALL_CLASS
7089,489687,2649622-2003-00267,M,Y,N,*,03/06/2003,*,*,N,...,*,*,6944,LWS,NaN,Y,2003,MEDTRONIC,*,0
7514,507667,2649622-2003-00355,M,Y,N,*,03/13/2003,*,*,*,...,*,*,*,LWS,NaN,*,2003,MEDTRONIC,*,0
17701,687828,2649622-2006-00441,M,Y,Y,N,11/18/2005,001,Y,N,...,"DEFIBRILLATOR, AUTOMATIC IMPLANTABLE CARDIOVERTER",0LP,6949,LWS,8 MO,Y,2006,MEDTRONIC,SPRINT FIDELIS,0
19469,734998,2649622-2006-00783,M,Y,Y,N,05/23/2006,600,N,N,...,"DEFIBRILLATOR, AUTOMATIC IMPLANTABLE CARDIOVERTER",0LP,6931,LWS,12 MO,Y,2006,MEDTRONIC,SPRINT FIDELIS,0
27519,849636,2649622-2007-01218,M,Y,Y,Y,03/13/2007,305,Y,N,...,"DEFIBRILLATOR, AUTOMATIC IMPLANTABLE CARDIOVERTER",0LP,6949,LWS,24 MO,*,2007,MEDTRONIC,SPRINT FIDELIS,0


In [403]:
# identifying products in Appendix E of Dr. Ensign's thesis

# create a list of all the product dfs
productDFs = []

In [404]:
MASDEV["BIOTRONIK_AROX"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('AROX', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_AROX = MASDEV[MASDEV["BIOTRONIK_AROX"] == True]
MASDEV_BIOTRONIK_AROX = MASDEV_BIOTRONIK_AROX.assign(STD_DEVICE='BIOTRONIK_AROX')
# productDFs.append(MASDEV_BIOTRONIK_AROX)
MASDEV2 = MASDEV_BIOTRONIK_AROX
MASDEV_BIOTRONIK_AROX.shape

(6, 85)

In [405]:
MASDEV["BIOTRONIK_COROX_OTW"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('COROX OTW', regex=False) &
                                             MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_COROX_OTW = MASDEV[MASDEV["BIOTRONIK_COROX_OTW"] == True]
MASDEV_BIOTRONIK_COROX_OTW = MASDEV_BIOTRONIK_COROX_OTW.assign(STD_DEVICE='BIOTRONIK_COROX_OTW')
productDFs.append(MASDEV_BIOTRONIK_COROX_OTW)
MASDEV_BIOTRONIK_COROX_OTW.shape

(49, 85)

In [406]:
MASDEV["BIOTRONIK_KAINOX"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('KAINOX', regex=False) &
                                          MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_KAINOX = MASDEV[MASDEV["BIOTRONIK_KAINOX"] == True]
MASDEV_BIOTRONIK_KAINOX = MASDEV_BIOTRONIK_KAINOX.assign(STD_DEVICE='BIOTRONIK_KAINOX')
productDFs.append(MASDEV_BIOTRONIK_KAINOX)
MASDEV_BIOTRONIK_KAINOX.shape

(80, 85)

In [407]:
MASDEV["BIOTRONIK_KENTROX"] = pd.DataFrame((MASDEV['BRAND_NAME'].str.contains('KENTROX RV', regex=False) | 
                                            MASDEV['BRAND_NAME'].str.contains('KENTROX SL', regex=False)) & 
                                           (MASDEV['BRAND_NAME'].str.contains('STER', regex=False) == False) & 
                                           (MASDEV["BRAND_NAME"].str.contains("SEROID", regex=False) == False) &
                                           MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_KENTROX = MASDEV[MASDEV["BIOTRONIK_KENTROX"] == True]
MASDEV_BIOTRONIK_KENTROX = MASDEV_BIOTRONIK_KENTROX.assign(STD_DEVICE='BIOTRONIK_KENTROX')
productDFs.append(MASDEV_BIOTRONIK_KENTROX)
MASDEV_BIOTRONIK_KENTROX.shape

(102, 85)

In [408]:
MASDEV["BIOTRONIK_KENTROX_STEROID"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('KENTROX', regex=False) & 
                                                   (MASDEV['BRAND_NAME'].str.contains('STER', regex=False) | 
                                                    MASDEV['BRAND_NAME'].str.contains('SEROID', regex=False)) & 
                                                   MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_KENTROX_STEROID = MASDEV[MASDEV["BIOTRONIK_KENTROX_STEROID"] == True]
MASDEV_BIOTRONIK_KENTROX_STEROID = MASDEV_BIOTRONIK_KENTROX_STEROID.assign(STD_DEVICE='BIOTRONIK_KENTROX_STEROID')
productDFs.append(MASDEV_BIOTRONIK_KENTROX_STEROID)
MASDEV_BIOTRONIK_KENTROX_STEROID.shape

(305, 85)

In [409]:
MASDEV["BIOTRONIK_LINOX"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('LINOX', regex=False) & 
                                         (MASDEV['BRAND_NAME'].str.contains('SMAR', regex=False) == False) & 
                                         (MASDEV['BRAND_NAME'].str.contains(' SD', regex=False) |
                                          MASDEV['BRAND_NAME'].str.contains(' TD', regex=False) |
                                          MASDEV['BRAND_NAME'].str.contains(' S', regex=False) |
                                          MASDEV['BRAND_NAME'].str.contains(' T', regex=False)) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_LINOX = MASDEV[MASDEV["BIOTRONIK_LINOX"] == True]
MASDEV_BIOTRONIK_LINOX = MASDEV_BIOTRONIK_LINOX.assign(STD_DEVICE='BIOTRONIK_LINOX')
productDFs.append(MASDEV_BIOTRONIK_LINOX)
MASDEV_BIOTRONIK_LINOX.shape

(3711, 85)

In [410]:
MASDEV["BIOTRONIK_LINOX_SMART"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('LINOX', regex=False) & 
                                            MASDEV['BRAND_NAME'].str.contains('SMART', regex=False) & 
                                             (MASDEV['BRAND_NAME'].str.contains(' SD', regex=False) |
                                              MASDEV['BRAND_NAME'].str.contains(' S DX', regex=False) |
                                              MASDEV['BRAND_NAME'].str.contains(' TD', regex=False) |
                                              MASDEV['BRAND_NAME'].str.contains(' S', regex=False) |
                                              MASDEV['BRAND_NAME'].str.contains(' T', regex=False)) &
                                            (MASDEV['BRAND_NAME'].str.contains('PROM', regex=False) == False) &
                                            MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_LINOX_SMART = MASDEV[MASDEV["BIOTRONIK_LINOX_SMART"] == True]
MASDEV_BIOTRONIK_LINOX_SMART = MASDEV_BIOTRONIK_LINOX_SMART.assign(STD_DEVICE='BIOTRONIK_LINOX_SMART')
productDFs.append(MASDEV_BIOTRONIK_LINOX_SMART)
MASDEV_BIOTRONIK_LINOX_SMART.shape

(2998, 85)

In [411]:
MASDEV["BIOTRONIK_LINOX_SMART_PROMRI"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('LINOX', regex=False) & 
                                            MASDEV['BRAND_NAME'].str.contains('SMART', regex=False) & 
                                             (MASDEV['BRAND_NAME'].str.contains(' SD', regex=False) |
                                              MASDEV['BRAND_NAME'].str.contains(' S DX', regex=False) |
                                              MASDEV['BRAND_NAME'].str.contains(' DF4 SD', regex=False) |
                                              MASDEV['BRAND_NAME'].str.contains(' S', regex=False)) &
                                            MASDEV['BRAND_NAME'].str.contains('PROM', regex=False) &
                                            MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_LINOX_SMART_PROMRI = MASDEV[MASDEV["BIOTRONIK_LINOX_SMART_PROMRI"] == True]
MASDEV_BIOTRONIK_LINOX_SMART_PROMRI = MASDEV_BIOTRONIK_LINOX_SMART_PROMRI.assign(STD_DEVICE='BIOTRONIK_LINOX_SMART_PROMRI')
productDFs.append(MASDEV_BIOTRONIK_LINOX_SMART_PROMRI)
MASDEV_BIOTRONIK_LINOX_SMART_PROMRI.shape

(1074, 85)

In [412]:
MASDEV["BIOTRONIK_LUMAX"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('LUMAX', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_LUMAX = MASDEV[MASDEV["BIOTRONIK_LUMAX"] == True]
MASDEV_BIOTRONIK_LUMAX = MASDEV_BIOTRONIK_LUMAX.assign(STD_DEVICE='BIOTRONIK_LUMAX')
productDFs.append(MASDEV_BIOTRONIK_LUMAX)
MASDEV_BIOTRONIK_LUMAX.shape

(2201, 85)

In [413]:
MASDEV["BIOTRONIK_SELOX"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SELOX', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_SELOX = MASDEV[MASDEV["BIOTRONIK_SELOX"] == True]
MASDEV_BIOTRONIK_SELOX = MASDEV_BIOTRONIK_SELOX.assign(STD_DEVICE='BIOTRONIK_SELOX')
productDFs.append(MASDEV_BIOTRONIK_SELOX)
MASDEV_BIOTRONIK_SELOX.shape

(35, 85)

In [414]:
'''
MASDEV["BIOTRONIK_TILDA-R"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('TILDA-R', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_TILDA_R = MASDEV[MASDEV["BIOTRONIK_TILDA-R"] == True]
MASDEV_BIOTRONIK_TILDA_R = MASDEV_BIOTRONIK_TILDA_R.assign(STD_DEVICE='BIOTRONIK_TILDA-R')
productDFs.append(MASDEV_BIOTRONIK_TILDA_R)
MASDEV_BIOTRONIK_TILDA_R.shape
'''
# no MDRs were found

'\nMASDEV["BIOTRONIK_TILDA-R"] = pd.DataFrame(MASDEV[\'BRAND_NAME\'].str.contains(\'TILDA-R\', regex=False) &\n                                        MASDEV[\'STD_MANUFACTURER\'].str.contains(\'BIOTRONIK\', regex=False))\nMASDEV_BIOTRONIK_TILDA_R = MASDEV[MASDEV["BIOTRONIK_TILDA-R"] == True]\nMASDEV_BIOTRONIK_TILDA_R = MASDEV_BIOTRONIK_TILDA_R.assign(STD_DEVICE=\'BIOTRONIK_TILDA-R\')\nproductDFs.append(MASDEV_BIOTRONIK_TILDA_R)\nMASDEV_BIOTRONIK_TILDA_R.shape\n'

In [415]:
MASDEV["BIOTRONIK_VIGILA_1"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('VIGILA 1', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_VIGILA_1 = MASDEV[MASDEV["BIOTRONIK_VIGILA_1"] == True]
MASDEV_BIOTRONIK_VIGILA_1 = MASDEV_BIOTRONIK_VIGILA_1.assign(STD_DEVICE='BIOTRONIK_VIGILA_1')
productDFs.append(MASDEV_BIOTRONIK_VIGILA_1)
MASDEV_BIOTRONIK_VIGILA_1.shape

(108, 85)

In [416]:
MASDEV["BIOTRONIK_VIGILA_2"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('VIGILA 2', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BIOTRONIK', regex=False))
MASDEV_BIOTRONIK_VIGILA_2 = MASDEV[MASDEV["BIOTRONIK_VIGILA_2"] == True]
MASDEV_BIOTRONIK_VIGILA_2 = MASDEV_BIOTRONIK_VIGILA_2.assign(STD_DEVICE='BIOTRONIK_VIGILA_2')
productDFs.append(MASDEV_BIOTRONIK_VIGILA_2)
MASDEV_BIOTRONIK_VIGILA_2.shape

(86, 85)

In [417]:
MASDEV["BOS_SCI_ACUITY"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('ACUITY', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_ACUITY = MASDEV[MASDEV["BOS_SCI_ACUITY"] == True]
MASDEV_BOS_SCI_ACUITY = MASDEV_BOS_SCI_ACUITY.assign(STD_DEVICE='BOS_SCI_ACUITY')
productDFs.append(MASDEV_BOS_SCI_ACUITY)
MASDEV_BOS_SCI_ACUITY.shape

(510, 85)

In [418]:
MASDEV["BOS_SCI_EASYTRAK"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('EASYTRAK', regex=False) &
                                          (MASDEV['BRAND_NAME'].str.contains(' 2', regex=False) == False) &
                                          (MASDEV['BRAND_NAME'].str.contains(' 3', regex=False) == False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_EASYTRAK = MASDEV[MASDEV["BOS_SCI_EASYTRAK"] == True]
MASDEV_BOS_SCI_EASYTRAK = MASDEV_BOS_SCI_EASYTRAK.assign(STD_DEVICE='BOS_SCI_EASYTRAK')
productDFs.append(MASDEV_BOS_SCI_EASYTRAK)
MASDEV_BOS_SCI_EASYTRAK.shape

(591, 85)

In [419]:
MASDEV["BOS_SCI_EASYTRAK2"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('EASYTRAK', regex=False) &
                                          MASDEV['BRAND_NAME'].str.contains(' 2', regex=False) &
                                          (MASDEV['BRAND_NAME'].str.contains(' 3', regex=False) == False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_EASYTRAK2 = MASDEV[MASDEV["BOS_SCI_EASYTRAK2"] == True]
MASDEV_BOS_SCI_EASYTRAK2 = MASDEV_BOS_SCI_EASYTRAK2.assign(STD_DEVICE='BOS_SCI_EASYTRAK2')
productDFs.append(MASDEV_BOS_SCI_EASYTRAK2)
MASDEV_BOS_SCI_EASYTRAK2.shape

(2361, 85)

In [420]:
MASDEV["BOS_SCI_EASYTRAK3"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('EASYTRAK', regex=False) &
                                          (MASDEV['BRAND_NAME'].str.contains(' 2', regex=False) == False) &
                                          MASDEV['BRAND_NAME'].str.contains(' 3', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_EASYTRAK3 = MASDEV[MASDEV["BOS_SCI_EASYTRAK3"] == True]
MASDEV_BOS_SCI_EASYTRAK3 = MASDEV_BOS_SCI_EASYTRAK3.assign(STD_DEVICE='BOS_SCI_EASYTRAK3')
productDFs.append(MASDEV_BOS_SCI_EASYTRAK3)
MASDEV_BOS_SCI_EASYTRAK3.shape

(132, 85)

In [421]:
MASDEV["BOS_SCI_ENDOTAK"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('ENDOTAK', regex=False) &
                                        (MASDEV["BRAND_NAME"].str.contains(" ENDURANCE", regex=False) == False) &
                                        (MASDEV["BRAND_NAME"].str.contains(" DSP", regex=False) == False) &
                                        (MASDEV["BRAND_NAME"].str.contains(" RELIANCE", regex=False) == False) &
                                        (MASDEV["BRAND_NAME"].str.contains(" SQ", regex=False) == False) &
                                        (MASDEV["BRAND_NAME"].str.contains(" C", regex=False) |
                                         MASDEV["BRAND_NAME"].str.contains("60", regex=False) |
                                         MASDEV["BRAND_NAME"].str.contains("70", regex=False)) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_ENDOTAK = MASDEV[MASDEV["BOS_SCI_ENDOTAK"] == True]
MASDEV_BOS_SCI_ENDOTAK = MASDEV_BOS_SCI_ENDOTAK.assign(STD_DEVICE='BOS_SCI_ENDOTAK')
productDFs.append(MASDEV_BOS_SCI_ENDOTAK)
MASDEV_BOS_SCI_ENDOTAK.shape

(638, 85)

In [422]:
MASDEV["BOS_SCI_ENDOTAK_ENDURANCE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('ENDOTAK', regex=False) &
                                        (MASDEV["BRAND_NAME"].str.contains("ENDURANCE", regex=False)) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_ENDOTAK_ENDURANCE = MASDEV[MASDEV["BOS_SCI_ENDOTAK_ENDURANCE"] == True]
MASDEV_BOS_SCI_ENDOTAK_ENDURANCE = MASDEV_BOS_SCI_ENDOTAK_ENDURANCE.assign(STD_DEVICE='BOS_SCI_ENDOTAK_ENDURANCE')
productDFs.append(MASDEV_BOS_SCI_ENDOTAK_ENDURANCE)
MASDEV_BOS_SCI_ENDOTAK_ENDURANCE.shape

(1901, 85)

In [423]:
MASDEV["BOS_SCI_ENDOTAK_DSP"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('ENDOTAK', regex=False) &
                                        (MASDEV["BRAND_NAME"].str.contains("DSP", regex=False)) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_ENDOTAK_DSP = MASDEV[MASDEV["BOS_SCI_ENDOTAK_DSP"] == True]
MASDEV_BOS_SCI_ENDOTAK_DSP = MASDEV_BOS_SCI_ENDOTAK_DSP.assign(STD_DEVICE='BOS_SCI_ENDOTAK_DSP')
productDFs.append(MASDEV_BOS_SCI_ENDOTAK_DSP)
MASDEV_BOS_SCI_ENDOTAK_DSP.shape

(1277, 85)

In [424]:
MASDEV["BOS_SCI_ENDOTAK_RELIANCE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('ENDOTAK', regex=False) &
                                        (MASDEV["BRAND_NAME"].str.contains("RELIANCE", regex=False)) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_ENDOTAK_RELIANCE = MASDEV[MASDEV["BOS_SCI_ENDOTAK_RELIANCE"] == True]
MASDEV_BOS_SCI_ENDOTAK_RELIANCE = MASDEV_BOS_SCI_ENDOTAK_RELIANCE.assign(STD_DEVICE='BOS_SCI_ENDOTAK_RELIANCE')
productDFs.append(MASDEV_BOS_SCI_ENDOTAK_RELIANCE)
MASDEV_BOS_SCI_ENDOTAK_RELIANCE.shape

(29354, 85)

In [425]:
MASDEV["BOS_SCI_ENDOTAK_SQ"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('ENDOTAK', regex=False) &
                                        (MASDEV["BRAND_NAME"].str.contains("SQ", regex=False)) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_ENDOTAK_SQ = MASDEV[MASDEV["BOS_SCI_ENDOTAK_SQ"] == True]
MASDEV_BOS_SCI_ENDOTAK_SQ = MASDEV_BOS_SCI_ENDOTAK_SQ.assign(STD_DEVICE='BOS_SCI_ENDOTAK_SQ')
productDFs.append(MASDEV_BOS_SCI_ENDOTAK_SQ)
MASDEV_BOS_SCI_ENDOTAK_SQ.shape

(110, 85)

In [426]:
MASDEV["BOS_SCI_FINELINE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('FINELINE', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_FINELINE = MASDEV[MASDEV["BOS_SCI_FINELINE"] == True]
MASDEV_BOS_SCI_FINELINE = MASDEV_BOS_SCI_FINELINE.assign(STD_DEVICE='BOS_SCI_FINELINE')
productDFs.append(MASDEV_BOS_SCI_FINELINE)
MASDEV_BOS_SCI_FINELINE.shape

(5, 85)

In [427]:
MASDEV["BOS_SCI_FLEXTEND"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('FLEXTEND', regex=False) &
                                            (MASDEV["BRAND_NAME"].str.contains(" 2", regex=False) == False) &
                                            (MASDEV["BRAND_NAME"].str.contains(" II", regex=False) == False) &
                                            MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_FLEXTEND = MASDEV[MASDEV["BOS_SCI_FLEXTEND"] == True]
MASDEV_BOS_SCI_FLEXTEND = MASDEV_BOS_SCI_FLEXTEND.assign(STD_DEVICE='BOS_SCI_FLEXTEND')
productDFs.append(MASDEV_BOS_SCI_FLEXTEND)
MASDEV_BOS_SCI_FLEXTEND.shape

(5, 85)

In [428]:
MASDEV["BOS_SCI_FLEXTEND2"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('FLEXTEND', regex=False) &
                                            (MASDEV["BRAND_NAME"].str.contains(" 2", regex=False) |
                                            MASDEV["BRAND_NAME"].str.contains(" II", regex=False)) &
                                            MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_FLEXTEND2 = MASDEV[MASDEV["BOS_SCI_FLEXTEND2"] == True]
MASDEV_BOS_SCI_FLEXTEND2 = MASDEV_BOS_SCI_FLEXTEND2.assign(STD_DEVICE='BOS_SCI_FLEXTEND2')
productDFs.append(MASDEV_BOS_SCI_FLEXTEND2)
MASDEV_BOS_SCI_FLEXTEND2.shape

(111, 85)

In [429]:
MASDEV["BOS_SCI_INTERVENE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('INTERVENE', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_INTERVENE = MASDEV[MASDEV["BOS_SCI_INTERVENE"] == True]
MASDEV_BOS_SCI_INTERVENE = MASDEV_BOS_SCI_INTERVENE.assign(STD_DEVICE='BOS_SCI_INTERVENE')
productDFs.append(MASDEV_BOS_SCI_INTERVENE)
MASDEV_BOS_SCI_INTERVENE.shape

(81, 85)

In [430]:
MASDEV["BOS_SCI_SELUTE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SELUTE', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_SELUTE = MASDEV[MASDEV["BOS_SCI_SELUTE"] == True]
MASDEV_BOS_SCI_SELUTE = MASDEV_BOS_SCI_SELUTE.assign(STD_DEVICE='BOS_SCI_SELUTE')
productDFs.append(MASDEV_BOS_SCI_SELUTE)
MASDEV_BOS_SCI_SELUTE.shape
# look more into "ATRIAL J SELUTE"

(475, 85)

In [431]:
'''
MASDEV["BOS_SCI_TACHY_LEAD_EPI_PATCH"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('TACHY', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('BOSTON SCIENTIFIC', regex=False))
MASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH = MASDEV[MASDEV["BOS_SCI_TACHY_LEAD_EPI_PATCH"] == True]
MASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH = MASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH.assign(STD_DEVICE='BOS_SCI_TACHY_LEAD_EPI_PATCH')
productDFs.append(MASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH)
MASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH.shape
'''
# no MDRs were found

'\nMASDEV["BOS_SCI_TACHY_LEAD_EPI_PATCH"] = pd.DataFrame(MASDEV[\'BRAND_NAME\'].str.contains(\'TACHY\', regex=False) &\n                                        MASDEV[\'STD_MANUFACTURER\'].str.contains(\'BOSTON SCIENTIFIC\', regex=False))\nMASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH = MASDEV[MASDEV["BOS_SCI_TACHY_LEAD_EPI_PATCH"] == True]\nMASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH = MASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH.assign(STD_DEVICE=\'BOS_SCI_TACHY_LEAD_EPI_PATCH\')\nproductDFs.append(MASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH)\nMASDEV_BOS_SCI_TACHY_LEAD_EPI_PATCH.shape\n'

In [432]:
MASDEV["CAMERON_HEALTH_QTRAK"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('Q-TRAK', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('CAMERON HEALTH', regex=False))
MASDEV_CAMERON_HEALTH_QTRAK = MASDEV[MASDEV["CAMERON_HEALTH_QTRAK"] == True]
MASDEV_CAMERON_HEALTH_QTRAK = MASDEV_CAMERON_HEALTH_QTRAK.assign(STD_DEVICE='CAMERON_HEALTH_QTRAK')
productDFs.append(MASDEV_CAMERON_HEALTH_QTRAK)
MASDEV_CAMERON_HEALTH_QTRAK.shape

(129, 85)

In [433]:
'''
MASDEV["MEDTRONIC_AESCULA"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('AESCULA', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_AESCULA = MASDEV[MASDEV["MEDTRONIC_AESCULA"] == True]
MASDEV_MEDTRONIC_AESCULA = MASDEV_MEDTRONIC_AESCULA.assign(STD_DEVICE='MEDTRONIC_AESCULA')
productDFs.append(MASDEV_MEDTRONIC_AESCULA)
MASDEV_MEDTRONIC_AESCULA.shape
'''
# no MDRs were found

'\nMASDEV["MEDTRONIC_AESCULA"] = pd.DataFrame(MASDEV[\'BRAND_NAME\'].str.contains(\'AESCULA\', regex=False) &\n                                        MASDEV[\'STD_MANUFACTURER\'].str.contains(\'MEDTRONIC\', regex=False))\nMASDEV_MEDTRONIC_AESCULA = MASDEV[MASDEV["MEDTRONIC_AESCULA"] == True]\nMASDEV_MEDTRONIC_AESCULA = MASDEV_MEDTRONIC_AESCULA.assign(STD_DEVICE=\'MEDTRONIC_AESCULA\')\nproductDFs.append(MASDEV_MEDTRONIC_AESCULA)\nMASDEV_MEDTRONIC_AESCULA.shape\n'

In [434]:
MASDEV["MEDTRONIC_ATTAIN"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('ATTAIN', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_ATTAIN = MASDEV[MASDEV["MEDTRONIC_ATTAIN"] == True]
MASDEV_MEDTRONIC_ATTAIN = MASDEV_MEDTRONIC_ATTAIN.assign(STD_DEVICE='MEDTRONIC_ATTAIN')
productDFs.append(MASDEV_MEDTRONIC_ATTAIN)
MASDEV_MEDTRONIC_ATTAIN.shape

(14, 85)

In [435]:
MASDEV["MEDTRONIC_CAPSURE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('CAPSURE', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_CAPSURE = MASDEV[MASDEV["MEDTRONIC_CAPSURE"] == True]
MASDEV_MEDTRONIC_CAPSURE = MASDEV_MEDTRONIC_CAPSURE.assign(STD_DEVICE='MEDTRONIC_CAPSURE')
productDFs.append(MASDEV_MEDTRONIC_CAPSURE)
MASDEV_MEDTRONIC_CAPSURE.shape

(354, 85)

In [436]:
MASDEV["MEDTRONIC_EPICARDIAL_PATCH"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('EPICARDIAL', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_EPICARDIAL_PATCH = MASDEV[MASDEV["MEDTRONIC_EPICARDIAL_PATCH"] == True]
MASDEV_MEDTRONIC_EPICARDIAL_PATCH = MASDEV_MEDTRONIC_EPICARDIAL_PATCH.assign(STD_DEVICE='MEDTRONIC_EPICARDIAL_PATCH')
productDFs.append(MASDEV_MEDTRONIC_EPICARDIAL_PATCH)
MASDEV_MEDTRONIC_EPICARDIAL_PATCH.shape

(97, 85)

In [437]:
MASDEV["MEDTRONIC_SELECTSECURE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SELECT', regex=False) &
                                                MASDEV["BRAND_NAME"].str.contains("SECURE", regex=False) &
                                                MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_SELECTSECURE = MASDEV[MASDEV["MEDTRONIC_SELECTSECURE"] == True]
MASDEV_MEDTRONIC_SELECTSECURE = MASDEV_MEDTRONIC_SELECTSECURE.assign(STD_DEVICE='MEDTRONIC_SELECTSECURE')
productDFs.append(MASDEV_MEDTRONIC_SELECTSECURE)
MASDEV_MEDTRONIC_SELECTSECURE.shape

(66, 85)

In [438]:
MASDEV["MEDTRONIC_SPRINT"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SPRINT', regex=False) &
                                          (MASDEV["BRAND_NAME"].str.contains("FID", regex=False) == False) &
                                          (MASDEV["BRAND_NAME"].str.contains("FIELIS", regex=False) == False) &
                                          (MASDEV["BRAND_NAME"].str.contains("FEDELIS", regex=False) == False) &
                                          ((MASDEV["BRAND_NAME"].str.contains(" QUAT", regex=False) == False) &
                                           (MASDEV["BRAND_NAME"].str.contains("QUADRO", regex=False) == False)) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_SPRINT = MASDEV[MASDEV["MEDTRONIC_SPRINT"] == True]
MASDEV_MEDTRONIC_SPRINT = MASDEV_MEDTRONIC_SPRINT.assign(STD_DEVICE='MEDTRONIC_SPRINT')
productDFs.append(MASDEV_MEDTRONIC_SPRINT)
MASDEV_MEDTRONIC_SPRINT.shape

(4634, 85)

In [439]:
MASDEV["MEDTRONIC_SPRINT_FIDELIS"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SPRINT', regex=False) &
                                                    MASDEV["BRAND_NAME"].str.contains("FID", regex=False) &
                                                    MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_SPRINT_FIDELIS = MASDEV[MASDEV["MEDTRONIC_SPRINT_FIDELIS"] == True]
MASDEV_MEDTRONIC_SPRINT_FIDELIS = MASDEV_MEDTRONIC_SPRINT_FIDELIS.assign(STD_DEVICE='MEDTRONIC_SPRINT_FIDELIS')
MASDEV_MEDTRONIC_SPRINT_FIDELIS = MASDEV_MEDTRONIC_SPRINT_FIDELIS.assign(RECALL_CLASS='1')
productDFs.append(MASDEV_MEDTRONIC_SPRINT_FIDELIS)
MASDEV_MEDTRONIC_SPRINT_FIDELIS.shape

(14339, 85)

In [440]:
MASDEV["MEDTRONIC_SPRINT_QUATTRO"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SPRINT', regex=False) &
                                                            MASDEV["BRAND_NAME"].str.contains("QU", regex=False) &
                                                            (MASDEV["BRAND_NAME"].str.contains("SECURE", regex=False) == False) &
                                                            MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_SPRINT_QUATTRO = MASDEV[MASDEV["MEDTRONIC_SPRINT_QUATTRO"] == True]
MASDEV_MEDTRONIC_SPRINT_QUATTRO = MASDEV_MEDTRONIC_SPRINT_QUATTRO.assign(STD_DEVICE='MEDTRONIC_SPRINT_QUATTRO')
productDFs.append(MASDEV_MEDTRONIC_SPRINT_QUATTRO)
MASDEV_MEDTRONIC_SPRINT_QUATTRO.shape

(6057, 85)

In [441]:
MASDEV["MEDTRONIC_SPRINT_QUATTRO_SECURE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SPRINT', regex=False) &
                                                            MASDEV["BRAND_NAME"].str.contains("QU", regex=False) &
                                                            MASDEV["BRAND_NAME"].str.contains("SECURE", regex=False)&
                                                            (MASDEV["BRAND_NAME"].str.contains(" MRI", regex=False) == False) &
                                                            MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE = MASDEV[MASDEV["MEDTRONIC_SPRINT_QUATTRO_SECURE"] == True]
MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE = MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE.assign(STD_DEVICE='MEDTRONIC_SPRINT_QUATTRO_SECURE')
MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE = MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE.assign(RECALL_CLASS='2')
productDFs.append(MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE)
MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE.shape

(38856, 85)

In [442]:
MASDEV["MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SPRINT', regex=False) &
                                                            MASDEV["BRAND_NAME"].str.contains("QU", regex=False) &
                                                            MASDEV["BRAND_NAME"].str.contains("SECURE", regex=False) &
                                                            MASDEV["BRAND_NAME"].str.contains(" MRI", regex=False) &
                                                            MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI = MASDEV[MASDEV["MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI"] == True]
MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI = MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI.assign(STD_DEVICE='MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI')
productDFs.append(MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI)
MASDEV_MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI.shape

(10462, 85)

In [443]:
MASDEV["MEDTRONIC_SUBQ"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SUB', regex=False) &
                                        MASDEV["BRAND_NAME"].str.contains("Q", regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_SUBQ = MASDEV[MASDEV["MEDTRONIC_SUBQ"] == True]
MASDEV_MEDTRONIC_SUBQ = MASDEV_MEDTRONIC_SUBQ.assign(STD_DEVICE='MEDTRONIC_SUBQ')
productDFs.append(MASDEV_MEDTRONIC_SUBQ)
MASDEV_MEDTRONIC_SUBQ.shape

(300, 85)

In [444]:
# SVC may be part of Transvene, or something other than a product entirely
# no information was found on the web, will look into this more
'''
MASDEV["MEDTRONIC_SVC"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SVC', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_SVC = MASDEV[MASDEV["MEDTRONIC_SVC"] == True]
MASDEV_MEDTRONIC_SVC = MASDEV_MEDTRONIC_SVC.assign(STD_DEVICE='MEDTRONIC_SVC')
productDFs.append(productDFs)
MASDEV_MEDTRONIC_SVC.shape
'''

'\nMASDEV["MEDTRONIC_SVC"] = pd.DataFrame(MASDEV[\'BRAND_NAME\'].str.contains(\'SVC\', regex=False) &\n                                        MASDEV[\'STD_MANUFACTURER\'].str.contains(\'MEDTRONIC\', regex=False))\nMASDEV_MEDTRONIC_SVC = MASDEV[MASDEV["MEDTRONIC_SVC"] == True]\nMASDEV_MEDTRONIC_SVC = MASDEV_MEDTRONIC_SVC.assign(STD_DEVICE=\'MEDTRONIC_SVC\')\nproductDFs.append(productDFs)\nMASDEV_MEDTRONIC_SVC.shape\n'

In [445]:
MASDEV["MEDTRONIC_TRANSVENE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('TRANSVENE', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('MEDTRONIC', regex=False))
MASDEV_MEDTRONIC_TRANSVENE = MASDEV[MASDEV["MEDTRONIC_TRANSVENE"] == True]
MASDEV_MEDTRONIC_TRANSVENE = MASDEV_MEDTRONIC_TRANSVENE.assign(STD_DEVICE='MEDTRONIC_TRANSVENE')
productDFs.append(MASDEV_MEDTRONIC_TRANSVENE)
MASDEV_MEDTRONIC_TRANSVENE.shape

(2239, 85)

In [446]:
MASDEV[MASDEV["STD_MANUFACTURER"] == "SORIN"]["BRAND_NAME"].value_counts()
# no MDRs for Sorin Angeflex, Sorin Angepass

PARADYM                          1243
OVATIO                            662
PLATINIUM                         653
ISOLINE                           563
PARADYM RF                        226
ALTO                              212
ORCHESTRA                         136
ALTO 2                             74
PARADYM 2                          66
ISOLINE 2CR6                       54
INTENSIA                           46
SMARTVIEW                          44
SWIFT                              20
DEFENDER IV                        14
ORCHESTRA PLUS LINK                11
SMARTVIEW HOTSPOT                  11
SITUS                              10
ISOLINE 2CT6                       10
OVATIO CRT                         10
STELID II                           7
REPLY                               7
SITUS OTW                           6
KORA                                6
PARADYM CRT                         5
REPLY 200                           3
SYMPHONY                            3
STELIX II   

In [447]:
MASDEV["SORIN_ISOLINE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('ISOLINE', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('SORIN', regex=False))
MASDEV_SORIN_ISOLINE = MASDEV[MASDEV["SORIN_ISOLINE"] == True]
MASDEV_SORIN_ISOLINE = MASDEV_SORIN_ISOLINE.assign(STD_DEVICE='SORIN_ISOLINE')
MASDEV_SORIN_ISOLINE = MASDEV_SORIN_ISOLINE.assign(RECALL_CLASS="2")
productDFs.append(MASDEV_SORIN_ISOLINE)
MASDEV_SORIN_ISOLINE.shape
# no results were found for "ISOLENE", but were found for "ISOLINE"
# assuming that this is a typo on Dr. Ensign's part

(627, 85)

In [448]:
MASDEV["SORIN_SITUS"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SITUS', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('SORIN', regex=False))
MASDEV_SORIN_SITUS = MASDEV[MASDEV["SORIN_SITUS"] == True]
MASDEV_SORIN_SITUS = MASDEV_SORIN_SITUS.assign(STD_DEVICE='SORIN_SITUS')
productDFs.append(MASDEV_SORIN_SITUS)
MASDEV_SORIN_SITUS.shape

(16, 85)

In [449]:
MASDEV["SORIN_STELID_II"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('STELID', regex=False) &
                                         MASDEV["BRAND_NAME"].str.contains("II", regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('SORIN', regex=False))
MASDEV_SORIN_STELID_II = MASDEV[MASDEV["SORIN_STELID_II"] == True]
MASDEV_SORIN_STELID_II = MASDEV_SORIN_STELID_II.assign(STD_DEVICE='SORIN_STELID_II')
productDFs.append(MASDEV_SORIN_STELID_II)
MASDEV_SORIN_STELID_II.shape

(7, 85)

In [450]:
MASDEV["SORIN_STELIX_II"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('STELIX', regex=False) &
                                         MASDEV["BRAND_NAME"].str.contains("II", regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('SORIN', regex=False))
MASDEV_SORIN_STELIX_II = MASDEV[MASDEV["SORIN_STELIX_II"] == True]
MASDEV_SORIN_STELIX_II = MASDEV_SORIN_STELIX_II.assign(STD_DEVICE='SORIN_STELIX_II')
productDFs.append(MASDEV_SORIN_STELIX_II)
MASDEV_SORIN_STELIX_II.shape

(3, 85)

In [451]:
MASDEV["STJUDE_DURATA"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('DURATA', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_DURATA = MASDEV[MASDEV["STJUDE_DURATA"] == True]
MASDEV_STJUDE_DURATA = MASDEV_STJUDE_DURATA.assign(STD_DEVICE='STJUDE_DURATA')
productDFs.append(MASDEV_STJUDE_DURATA)
MASDEV_STJUDE_DURATA.shape

(26277, 85)

In [452]:
MASDEV["STJUDE_QUARTET"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('QUARTET', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_QUARTET = MASDEV[MASDEV["STJUDE_QUARTET"] == True]
MASDEV_STJUDE_QUARTET = MASDEV_STJUDE_QUARTET.assign(STD_DEVICE='STJUDE_QUARTET')
productDFs.append(MASDEV_STJUDE_QUARTET)
MASDEV_STJUDE_QUARTET.shape

(2029, 85)

In [453]:
MASDEV["STJUDE_QUICKFLEX"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('QUICKFLEX', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_QUICKFLEX = MASDEV[MASDEV["STJUDE_QUICKFLEX"] == True]
MASDEV_STJUDE_QUICKFLEX = MASDEV_STJUDE_QUICKFLEX.assign(STD_DEVICE='STJUDE_QUICKFLEX')
productDFs.append(MASDEV_STJUDE_QUICKFLEX)
MASDEV_STJUDE_QUICKFLEX.shape
# no entries for St. Jude Quickflex u were found

(1382, 85)

In [454]:
MASDEV["STJUDE_QUICKSITE"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('QUICKSITE', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_QUICKSITE = MASDEV[MASDEV["STJUDE_QUICKSITE"] == True]
MASDEV_STJUDE_QUICKSITE = MASDEV_STJUDE_QUICKSITE.assign(STD_DEVICE='STJUDE_QUICKSITE')
productDFs.append(MASDEV_STJUDE_QUICKSITE)
MASDEV_STJUDE_QUICKSITE.shape

(342, 85)

In [455]:
MASDEV["STJUDE_RIATA"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('RIATA', regex=False) &
                                      (MASDEV["BRAND_NAME"].str.contains("ST", regex=False) == False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_RIATA = MASDEV[MASDEV["STJUDE_RIATA"] == True]
MASDEV_STJUDE_RIATA = MASDEV_STJUDE_RIATA.assign(STD_DEVICE='STJUDE_RIATA')
MASDEV_STJUDE_RIATA = MASDEV_STJUDE_RIATA.assign(RECALL_CLASS="1")
productDFs.append(MASDEV_STJUDE_RIATA)
MASDEV_STJUDE_RIATA.shape

(17992, 85)

In [456]:
MASDEV["STJUDE_RIATA_ST"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('RIATA', regex=False) &
                                            MASDEV["BRAND_NAME"].str.contains("ST", regex=False) &
                                            (MASDEV["BRAND_NAME"].str.contains("OPTIM", regex=False) == False) &
                                            MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_RIATA_ST = MASDEV[MASDEV["STJUDE_RIATA_ST"] == True]
MASDEV_STJUDE_RIATA_ST = MASDEV_STJUDE_RIATA_ST.assign(STD_DEVICE='STJUDE_RIATA_ST')
MASDEV_STJUDE_RIATA_ST = MASDEV_STJUDE_RIATA_ST.assign(RECALL_CLASS="1")
productDFs.append(MASDEV_STJUDE_RIATA_ST)
MASDEV_STJUDE_RIATA_ST.shape

(6321, 85)

In [457]:
MASDEV["STJUDE_RIATA_ST_OPTIM"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('RIATA', regex=False) &
                                                MASDEV["BRAND_NAME"].str.contains("ST", regex=False) &
                                                MASDEV["BRAND_NAME"].str.contains("OPTIM", regex=False) &
                                                MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_RIATA_ST_OPTIM = MASDEV[MASDEV["STJUDE_RIATA_ST_OPTIM"] == True]
MASDEV_STJUDE_RIATA_ST_OPTIM = MASDEV_STJUDE_RIATA_ST_OPTIM.assign(STD_DEVICE='STJUDE_RIATA_ST_OPTIM')
productDFs.append(MASDEV_STJUDE_RIATA_ST_OPTIM)
MASDEV_STJUDE_RIATA_ST_OPTIM.shape

(2435, 85)

In [458]:
MASDEV["STJUDE_SPL"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('SPL', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_SPL = MASDEV[MASDEV["STJUDE_SPL"] == True]
MASDEV_STJUDE_SPL = MASDEV_STJUDE_SPL.assign(STD_DEVICE='STJUDE_SPL')
productDFs.append(MASDEV_STJUDE_SPL)
MASDEV_STJUDE_SPL.shape

(479, 85)

In [459]:
MASDEV["STJUDE_TENDRIL"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('TENDRIL', regex=False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_TENDRIL = MASDEV[MASDEV["STJUDE_TENDRIL"] == True]
MASDEV_STJUDE_TENDRIL = MASDEV_STJUDE_TENDRIL.assign(STD_DEVICE='STJUDE_TENDRIL')
productDFs.append(MASDEV_STJUDE_TENDRIL)
MASDEV_STJUDE_TENDRIL.shape

(11, 85)

In [460]:
MASDEV["STJUDE_TVL"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('TVL', regex=False) &
                                        (MASDEV["BRAND_NAME"].str.contains("ADX", regex=False) == False) &
                                        (MASDEV["BRAND_NAME"].str.contains("ADC", regex=False) == False) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_TVL = MASDEV[MASDEV["STJUDE_TVL"] == True]
MASDEV_STJUDE_TVL = MASDEV_STJUDE_TVL.assign(STD_DEVICE='STJUDE_TVL')
productDFs.append(MASDEV_STJUDE_TVL)
MASDEV_STJUDE_TVL.shape

(125, 85)

In [461]:
MASDEV["STJUDE_TVL_ADX"] = pd.DataFrame(MASDEV['BRAND_NAME'].str.contains('TVL', regex=False) &
                                        (MASDEV["BRAND_NAME"].str.contains("ADX", regex=False) |
                                         MASDEV["BRAND_NAME"].str.contains("ADC", regex=False)) &
                                        MASDEV['STD_MANUFACTURER'].str.contains('ST. JUDE', regex=False))
MASDEV_STJUDE_TVL_ADX = MASDEV[MASDEV["STJUDE_TVL_ADX"] == True]
MASDEV_STJUDE_TVL_ADX = MASDEV_STJUDE_TVL_ADX.assign(STD_DEVICE='STJUDE_TVL_ADX')
productDFs.append(MASDEV_STJUDE_TVL_ADX)
MASDEV_STJUDE_TVL_ADX.shape

(440, 85)

In [462]:
# "moving" cell that I used to check the accepted values for each product df
MASDEV_STJUDE_TVL_ADX["BRAND_NAME"].value_counts()

TVL ADX                                                         218
TVL-ADX ACTIVE-FIXATION STEROID-ELUTING                         112
TVL-ADX ACTIVE FIXATION LEAD                                     38
TVL-ADX ACTIVE-FIXATION STEROID-ELUTING TACHYARRHYTHMIA          37
TVL-ADX ACTIVE-FIXATION STEROID-ELUTING TACHYARRHYTHMIA LEAD     15
TVL-ADX ACTIVE-FIXATION LEAD                                      4
TVL-ADX ACTIVE-FIXATION STEROID ELUTING                           3
TVL-ADX ACTIVE-FIXATION STERIOD-ELUTING                           3
TVL-ADX ACTIVE FIXATION STEROID ELUTING TACHYARRHYTHMIA           2
TVL-ADX ACTIVE FIXATION STEROID-ELUTING TACHYARRHYTHMIA           2
TVL-ADX ACTIVE-FIXATION STEROID ELUTING TACHYARRHYTHMIA           1
TVL-ADX ACTIVE-FIXATION STERIOD-ELUTING TACHYARRHYTHMIA LEAD      1
TVL-ADC ACTIVE-FIXATION STEROID-ELUTING                           1
TVL-ADX ACTIVE-FIXATION STERIOID-ELUTING                          1
TVL-ADX 1559 ICD LEAD                           

In [463]:
# check if all dfs were added to productDFs
for df in productDFs:
        print(df["STD_DEVICE"].iloc[0])

BIOTRONIK_COROX_OTW
BIOTRONIK_KAINOX
BIOTRONIK_KENTROX
BIOTRONIK_KENTROX_STEROID
BIOTRONIK_LINOX
BIOTRONIK_LINOX_SMART
BIOTRONIK_LINOX_SMART_PROMRI
BIOTRONIK_LUMAX
BIOTRONIK_SELOX
BIOTRONIK_VIGILA_1
BIOTRONIK_VIGILA_2
BOS_SCI_ACUITY
BOS_SCI_EASYTRAK
BOS_SCI_EASYTRAK2
BOS_SCI_EASYTRAK3
BOS_SCI_ENDOTAK
BOS_SCI_ENDOTAK_ENDURANCE
BOS_SCI_ENDOTAK_DSP
BOS_SCI_ENDOTAK_RELIANCE
BOS_SCI_ENDOTAK_SQ
BOS_SCI_FINELINE
BOS_SCI_FLEXTEND
BOS_SCI_FLEXTEND2
BOS_SCI_INTERVENE
BOS_SCI_SELUTE
CAMERON_HEALTH_QTRAK
MEDTRONIC_ATTAIN
MEDTRONIC_CAPSURE
MEDTRONIC_EPICARDIAL_PATCH
MEDTRONIC_SELECTSECURE
MEDTRONIC_SPRINT
MEDTRONIC_SPRINT_FIDELIS
MEDTRONIC_SPRINT_QUATTRO
MEDTRONIC_SPRINT_QUATTRO_SECURE
MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI
MEDTRONIC_SUBQ
MEDTRONIC_TRANSVENE
SORIN_ISOLINE
SORIN_SITUS
SORIN_STELID_II
SORIN_STELIX_II
STJUDE_DURATA
STJUDE_QUARTET
STJUDE_QUICKFLEX
STJUDE_QUICKSITE
STJUDE_RIATA
STJUDE_RIATA_ST
STJUDE_RIATA_ST_OPTIM
STJUDE_SPL
STJUDE_TENDRIL
STJUDE_TVL
STJUDE_TVL_ADX


In [464]:
# merge the dfs in productDFs

for df in productDFs:
    MASDEV2 = MASDEV2.append(df)
    print(MASDEV2.shape)

(55, 85)
(135, 85)
(237, 85)
(542, 85)
(4253, 85)
(7251, 85)
(8325, 85)
(10526, 85)
(10561, 85)
(10669, 85)
(10755, 85)
(11265, 85)
(11856, 85)
(14217, 85)
(14349, 85)
(14987, 85)
(16888, 85)
(18165, 85)
(47519, 85)
(47629, 85)
(47634, 85)
(47639, 85)
(47750, 85)
(47831, 85)
(48306, 85)
(48435, 85)
(48449, 85)
(48803, 85)
(48900, 85)
(48966, 85)
(53600, 85)
(67939, 85)
(73996, 85)
(112852, 85)
(123314, 85)
(123614, 85)
(125853, 85)
(126480, 85)
(126496, 85)
(126503, 85)
(126506, 85)
(152783, 85)
(154812, 85)
(156194, 85)
(156536, 85)
(174528, 85)
(180849, 85)
(183284, 85)
(183763, 85)
(183774, 85)
(183899, 85)
(184339, 85)


In [465]:
MASDEV2["STD_DEVICE"].value_counts()

MEDTRONIC_SPRINT_QUATTRO_SECURE        38856
BOS_SCI_ENDOTAK_RELIANCE               29354
STJUDE_DURATA                          26277
STJUDE_RIATA                           17992
MEDTRONIC_SPRINT_FIDELIS               14339
MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI    10462
STJUDE_RIATA_ST                         6321
MEDTRONIC_SPRINT_QUATTRO                6057
MEDTRONIC_SPRINT                        4634
BIOTRONIK_LINOX                         3711
BIOTRONIK_LINOX_SMART                   2998
STJUDE_RIATA_ST_OPTIM                   2435
BOS_SCI_EASYTRAK2                       2361
MEDTRONIC_TRANSVENE                     2239
BIOTRONIK_LUMAX                         2201
STJUDE_QUARTET                          2029
BOS_SCI_ENDOTAK_ENDURANCE               1901
STJUDE_QUICKFLEX                        1382
BOS_SCI_ENDOTAK_DSP                     1277
BIOTRONIK_LINOX_SMART_PROMRI            1074
BOS_SCI_ENDOTAK                          638
SORIN_ISOLINE                            627
BOS_SCI_EA

In [466]:
MASDEV2["RECALL_CLASS"].value_counts()

0    106204
2     39483
1     38652
Name: RECALL_CLASS, dtype: int64

In [467]:
# remove unwanted columns
print("the shape of MASDEV2 is: " + str(MASDEV2.shape))

keptCols = ['MDR_REPORT_KEY',
            'REPORT_NUMBER',
            'REPORT_SOURCE_CODE',
            'MANUFACTURER_LINK_FLAG_',
            'ADVERSE_EVENT_FLAG',
            'PRODUCT_PROBLEM_FLAG',
            'DATE_OF_EVENT',
            'REPORTER_OCCUPATION_CODE',
            'HEALTH_PROFESSIONAL',
            'INITIAL_REPORT_TO_FDA',
            'REPORT_TO_FDA',
            'EVENT_LOCATION',
            'EVENT_TYPE',
            'TYPE_OF_REPORT',
            'DATE_ADDED',
            'DATE_CHANGED',
            'REPORTER_COUNTRY_CODE',
            'SUMMARY_REPORT',
            'DEVICE_SEQUENCE_NO',
            'DATE_RECEIVED',
            'BRAND_NAME',
            'GENERIC_NAME',
            'DEVICE_OPERATOR',
            'MODEL_NUMBER',
            'DEVICE_REPORT_PRODUCT_CODE',
            'DEVICE_AGE_TEXT',
            'DEVICE_EVALUATED_BY_MANUFACTUR',
            'year',
            'STD_MANUFACTURER',
            'STD_DEVICE',
            'RECALL_CLASS']

MASDEV2 = MASDEV2[keptCols]
print("the shape of MASDEV2 after column removal is: " + str(MASDEV2.shape))
print(MASDEV2.columns)

the shape of MASDEV2 is: (184339, 85)
the shape of MASDEV2 after column removal is: (184339, 32)
Index(['MDR_REPORT_KEY', 'REPORT_NUMBER', 'REPORT_SOURCE_CODE',
       'MANUFACTURER_LINK_FLAG_', 'ADVERSE_EVENT_FLAG', 'PRODUCT_PROBLEM_FLAG',
       'DATE_OF_EVENT', 'REPORTER_OCCUPATION_CODE', 'HEALTH_PROFESSIONAL',
       'INITIAL_REPORT_TO_FDA', 'REPORT_TO_FDA', 'EVENT_LOCATION',
       'EVENT_TYPE', 'TYPE_OF_REPORT', 'DATE_ADDED', 'DATE_CHANGED',
       'REPORTER_COUNTRY_CODE', 'SUMMARY_REPORT', 'DEVICE_SEQUENCE_NO',
       'DEVICE_SEQUENCE_NO', 'DATE_RECEIVED', 'BRAND_NAME', 'GENERIC_NAME',
       'DEVICE_OPERATOR', 'MODEL_NUMBER', 'DEVICE_REPORT_PRODUCT_CODE',
       'DEVICE_AGE_TEXT', 'DEVICE_EVALUATED_BY_MANUFACTUR', 'year',
       'STD_MANUFACTURER', 'STD_DEVICE', 'RECALL_CLASS'],
      dtype='object')


In [468]:
MASDEV2.to_pickle("./3_MASTER_DEVICE_File_wSTDDEVICE.pkl")

In [469]:
# work from below here is related to selecting the right dates

In [470]:
MASDEV2.info()
# "DATE_OF_EVENT", "DATE_ADDED", "DATE_CHANGED", "DATE_RECEIVED" all have date-related information stored
# Dr. Ensign uses "DATE_RECEIVED", so I will use it here

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184339 entries, 28790 to 410124
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   MDR_REPORT_KEY                  184339 non-null  object
 1   REPORT_NUMBER                   184339 non-null  object
 2   REPORT_SOURCE_CODE              184339 non-null  object
 3   MANUFACTURER_LINK_FLAG_         184339 non-null  object
 4   ADVERSE_EVENT_FLAG              184245 non-null  object
 5   PRODUCT_PROBLEM_FLAG            184256 non-null  object
 6   DATE_OF_EVENT                   167218 non-null  object
 7   REPORTER_OCCUPATION_CODE        178744 non-null  object
 8   HEALTH_PROFESSIONAL             177395 non-null  object
 9   INITIAL_REPORT_TO_FDA           180453 non-null  object
 10  REPORT_TO_FDA                   137341 non-null  object
 11  EVENT_LOCATION                  147625 non-null  object
 12  EVENT_TYPE                

In [471]:
MASDEV2['DATE_RECEIVED']

28790     2007/07/31
30694     2007/08/14
30721     2007/08/22
31540     2007/09/17
31541     2007/09/17
             ...    
389592    2019/09/16
394059    2019/10/30
395628    2019/11/14
398013    2019/12/16
410124    2020/04/16
Name: DATE_RECEIVED, Length: 184339, dtype: object

In [472]:
print("MASDEV2.shape: " + str(MASDEV2.shape))

MASDEV2["Before2013"] = pd.DataFrame(MASDEV2['DATE_RECEIVED'].str.contains('2000', regex=False) |
                                        MASDEV2["DATE_RECEIVED"].str.contains("2001", regex=False) |
                                        MASDEV2['DATE_RECEIVED'].str.contains('2002', regex=False) |
                                        MASDEV2["DATE_RECEIVED"].str.contains("2003", regex=False) |
                                        MASDEV2['DATE_RECEIVED'].str.contains('2004', regex=False) |
                                        MASDEV2["DATE_RECEIVED"].str.contains("2005", regex=False) |
                                        MASDEV2['DATE_RECEIVED'].str.contains('2006', regex=False) |
                                        MASDEV2["DATE_RECEIVED"].str.contains("2007", regex=False) |
                                        MASDEV2['DATE_RECEIVED'].str.contains('2008', regex=False) |
                                        MASDEV2["DATE_RECEIVED"].str.contains("2009", regex=False) |
                                        MASDEV2['DATE_RECEIVED'].str.contains('2010', regex=False) |
                                        MASDEV2["DATE_RECEIVED"].str.contains("2011", regex=False) |
                                        MASDEV2['DATE_RECEIVED'].str.contains('2012', regex=False) |
                                        MASDEV2["DATE_RECEIVED"].str.contains("2013", regex=False)
                                    )
MASDEV2_Before2013 = MASDEV2[MASDEV2["Before2013"] == True]
print("MASDEV2_Before2013.shape: " + str(MASDEV2_Before2013.shape))

MASDEV2.shape: (184339, 32)
MASDEV2_Before2013.shape: (76433, 33)


In [473]:
MASDEV2_Before2013.to_pickle("./3_MASTER_DEVICE_File_wSTDDEVICE_2000to2013.pkl")